## 3.2 텐서 : 다차원 배열

### 3.2.2 텐서 만들기

In [1]:
import torch
a = torch.ones(3) # 크기가 3인 1차원 텐서 만들고 1로 채우기
a

tensor([1., 1., 1.])

In [2]:
a[1]

tensor(1.)

In [3]:
float(a[1])

1.0

In [4]:
a[2] = 2.0
a

tensor([1., 1., 2.])

### 3.2.3 텐서의 핵심
- 연속적인 메모리가 할당되고 뷰 제공
- 각 요소는 32비트(4바이트) float 타입

In [5]:
points = torch.zeros(6)
points

tensor([0., 0., 0., 0., 0., 0.])

In [6]:
# 원하는 값으로 덮어씀
points[0] = 4.0
points[2] = 6.0
points

tensor([4., 0., 6., 0., 0., 0.])

In [8]:
# 생성자에 파이썬 리스트 넘겨도 됨
points = torch.tensor([4.0, 1.0, 4.0, 5.0, 3.0, 2.0])
points

tensor([4., 1., 4., 5., 3., 2.])

In [9]:
float(points[0]), float(points[1])

(4.0, 1.0)

In [17]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 4.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 4.]])

In [18]:
points.shape # size()

torch.Size([3, 2])

In [19]:
points[0, 1]

tensor(1.)

In [20]:
points[0]

tensor([4., 1.])

In [16]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

## 3.3 텐서 인덱싱

In [23]:
some_list = list(range(6))
some_list

[0, 1, 2, 3, 4, 5]

In [24]:
some_list[:]

[0, 1, 2, 3, 4, 5]

In [26]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 4.0]])

points[1:]

tensor([[5., 3.],
        [2., 4.]])

In [29]:
points[1:, :]

tensor([[5., 3.],
        [2., 4.]])

In [30]:
points[1:, 0]

tensor([5., 2.])

In [31]:
points[None] # 길이가 1인 차원 추가 (= unsqueeze)

tensor([[[4., 1.],
         [5., 3.],
         [2., 4.]]])

## 3.4 이름이 있는 텐서

In [18]:
import torch

In [5]:
img_t = torch.randn(3, 5, 5) # 채널, rows, cols
weights = torch.tensor([0.2126, 0.7152, 0.0722]) # torch.Size([3])
batch_t = torch.randn(2, 3, 5, 5) # batch, channel, rows, cols

In [6]:
# 컬러 -> 흑백
img_gray_naive = img_t.mean(-3) # 채널의 인덱스 -3
batch_gray_naive = batch_t.mean(-3)

img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [13]:
# 브로드캐스팅
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1) # torch.Size([3, 1, 1])
img_weights = img_t * unsqueezed_weights # torch.Size([3, 5, 5])
batch_weights = batch_t * unsqueezed_weights # torch.Size([2, 3, 5, 5])

img_weighted = img_weights.sum(-3)
batch_weighted = batch_weights.sum(-3)

In [17]:
# 곱셈 후 합 구하기
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
img_gray_weighted_fancy.shape

torch.Size([5, 5])

In [20]:
# 차원에 이름 부여
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [22]:
# 텐서 먼저 만들고 나중에 이름 지정
# ...를 사용하면 다른 차원은 건드리지 않는다.
img_named = img_t.refine_names(..., 'channels', 'rows', 'cols')
batch_named = batch_t.refine_names(..., 'batch', 'channels', 'rows', 'cols')

print(img_named.shape, img_named.names)
print(batch_named.shape, batch_named.names)

torch.Size([3, 5, 5]) ('channels', 'rows', 'cols')
torch.Size([2, 3, 5, 5]) ('batch', 'channels', 'rows', 'cols')


In [25]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names
# 빠진 차원 채우고, 존재하는 차원을 올바른 순서로 바꿔줌
# 원래 torch.Size([3])이었던 weights

(torch.Size([3, 1, 1]), ('channels', 'rows', 'cols'))

In [28]:
# 차원 인수 허용하는 함수
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'cols'))

In [39]:
gray_named = (img_named[..., :3] * weights_named).sum('channels')
# 이름이 다른 차원 결합 : 오류 발생

RuntimeError: Error when attempting to broadcast dims ['channels', 'rows', 'cols'] and dims ['channels']: dim 'cols' and dim 'channels' are at the same position from the right but do not match.

## 3.5 텐서 요소 타입

### 3.5.1 dtype으로 숫자 타입 지정하기

- torch.float32, torch.float : 32비트 단정밀도 부동소수 <'default'>
- torch.float64 : 64비트 배정밀도 부동소수
- torch.float16 : 16비트 반정밀도 부동소수
  
  
- torch.int8 : 부호 있는 8비트 정수  
- torch.uint8 : 부호 없는 8비트 정수  
- torch.int16, torch.short : 부호 있는 16비트 정수  
- torch.int32, torch.int : 부호 있는 32비트 정수  
- torch.int64, torch.long : 부호 있는 64비트 정수

### 3.5.3 텐서의 dtype 속성 관리

In [41]:
double_points = torch.ones(10, 2, dtype=torch.double)
double_points.dtype

torch.float64

In [43]:
short_points = torch.tensor([[1, 2], [3, 4]], dtype = torch.short)
short_points.dtype

torch.int16

In [46]:
# 캐스팅 메소드
double_points = torch.zeros(10, 2).double()
double_points = torch.zeros(10, 2).to(torch.double)

double_points.dtype

torch.float64

In [47]:
# 여러 타입의 입력들이 연산을 거치며 서로 섞일 때, 자동으로 가장 큰 타입으로 만듬
points_64 = torch.rand(5, dtype=torch.double) # torch.float64
points_short = points_64.to(torch.short) # torch.int16
points_64 * points_short

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

## 3.6 텐서 API

In [49]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [53]:
a_t = a.transpose(0, 1)

## 3.7 텐서의 저장소 관점

### 3.7.1 저장 공간 인덱싱

In [54]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_34520\2923139482.py:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points.storage()


 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [56]:
points_storage = points.storage()
points_storage[2]

5.0

In [57]:
points_storage[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

### 3.7.2 저장된 값을 수정하기 : 텐서 내부 연산

In [58]:
a = torch.ones(3, 2)
a.zero_()  # 모든 요소를 바꿔줌 ; _는 기존 텐서의 내용을 바꿈
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

## 3.8 텐서 메타데이터 : 사이즈, 오프셋, 스트라이드
- 사이즈 : 텐서의 각 차원 별로 들어가는 요소의 수를 표시한 튜플
- 저장 공간에 대한 오프셋 : 텐서의 첫 번째 요소를 가리키는 색인 값과 동일
- 스트라이드 : 각 차원에서 다음 요소를 가리키고 싶을 때 실제 저장 공간상에서 몇 개의 요소를 건너뛰어야 하는지 알려줌

### 3.8.1 다른 텐서의 저장 공간에 대한 뷰 만들기

In [62]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset() # 첫 번째 원소(5.0)가 storage에 저장되어 있는 인덱스

2

In [63]:
second_point.size() # 1차원이므로 size 클래스는 하나의 요소를 가지는 객체

torch.Size([2])

In [64]:
second_point.shape

torch.Size([2])

In [67]:
points.stride() # storage_offset + stride[0] * i + stide[1] * j

(2, 1)

In [70]:
second_point.stride()

(1,)

In [72]:
second_points[0] = 10.0
points  # 저장공간 바꾸면 원래 텐서도 바뀜

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In [73]:
# 서브텐서를 새 텐서로 복제
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points # 바뀌지 않음

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

### 3.8.2 복사 없이 텐서 전치하기

In [19]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [79]:
# 두 텐서가 같은 공간을 가리키고 있는지 확인
id(points.storage()) == id(points_t.storage())

True

In [81]:
points.stride(), points_t.stride()

((2, 1), (1, 2))

### 3.8.3 더 높은 차원에서의 전치 연산

In [82]:
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)
transpose_t.shape

torch.Size([5, 4, 3])

In [85]:
some_t.stride(), transpose_t.stride()

((20, 5, 1), (1, 5, 20))

### 3.8.4 인접한(contiguous) 텐서
- 인접한 텐서는 값 순회 시 띄엄띄엄 참조하지 않기에 데이터 지역성 관점에서 CPU 메모리 접근 효율이 좋다.
- 텐서가 이미 인접한 상태라면 contiguous가 실제로 하는 일은 없고 성능에 지장을 주지도 않는다.

In [87]:
points.is_contiguous() # 연속된 텐서

True

In [88]:
points_t.is_contiguous()

False

In [92]:
points_t_cont = points_t.contiguous() # 값은 동일하나 배치와 스트라이드가 바뀜
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [94]:
points_t.stride(), points_t_cont.stride()

((1, 2), (3, 1))

## 3.9 텐서를 GPU로 옮기기

### 3.9.1 텐서 디바이스 속성 관리
** 나의 노트북은 cuda설치 불가능,, 외장 없어서..

In [5]:
import torch

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
print(torch.cuda.is_available())

False


In [ ]:
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device='cuda')

In [ ]:
points_gpu = points.to(device='cuda')

In [1]:
points_gpu = points.to(device='cuda:0') # gpu를 0번부터 시작

In [ ]:
points = 2 * points # CPU에서 실행하는 곱셈
points_gpu = 2 * points.to(device='cuda') # GPU에서 실행하는 곱셈

In [ ]:
# GPU에서의 연산은 CPU로 옮겨지지 않는다. 다시 CPU로 옮기려면
points_cpu = points_gpu.to(device='cpu')

In [ ]:
points_gpu = points.cuda() # 기본으로 0번 GPU에 복사
points_gpu = points.cuda(0)
points_cpu = points_gpu.cpu()

## 3.10 넘파이 호환

In [6]:
points = torch.ones(3, 4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [7]:
# 넘파이 -> 텐서
points = torch.from_numpy(points_np)
points

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

## 3.12 텐서 직렬화

In [8]:
# 우리의 points 텐서를 outpoints.t 파일에 저장
torch.save(points, r'C:\Users\knuyh\Desktop\민지\스터디\파이토치 딥러닝 마스터\Minji\Part01\CH03\Data\ourpoints.t')

In [9]:
with open(r'C:\Users\knuyh\Desktop\민지\스터디\파이토치 딥러닝 마스터\Minji\Part01\CH03\Data\ourpoints.t', 'wb') as f :
    torch.save(points, f)

In [10]:
# 불러들이기
points = torch.load(r'C:\Users\knuyh\Desktop\민지\스터디\파이토치 딥러닝 마스터\Minji\Part01\CH03\Data\ourpoints.t')

In [11]:
with open(r'C:\Users\knuyh\Desktop\민지\스터디\파이토치 딥러닝 마스터\Minji\Part01\CH03\Data\ourpoints.t', 'rb') as f :
    points = torch.load(f)

### 3.12.1 h5py로 HDF5 병렬화하기

In [12]:
import h5py

In [19]:
f = h5py.File(r'C:\Users\knuyh\Desktop\민지\스터디\파이토치 딥러닝 마스터\Minji\Part01\CH03\Data\ourpoints.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy()) # coords : 파일의 키
f.close()

In [23]:
# 데이터셋의 가장 뒤 포인트 두 개만 읽기
f = h5py.File(r'C:\Users\knuyh\Desktop\민지\스터디\파이토치 딥러닝 마스터\Minji\Part01\CH03\Data\ourpoints.hdf5', 'r')
dset = f['coords']
last_points = dset[-2:]
last_points

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [24]:
last_points = torch.from_numpy(dset[-2:]) # 곧바로 텐서 얻음
f.close()

In [25]:
last_points # close로 닫아도 텐서의 저장 공간으로 복사된 상태

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]])